# **Пункт 3 - Проведение расчётов и исследований**



В этом ноутбуке представлено решение 1, 6 и 7 задач из третьего пункта

# **3.1 Посчитать процент пациентов с хроническими заболеваниям (сахарный диабет, гипертония, хроническая болезнь почек) для групп “есть ОПП/нет ОПП”**

In [6]:
import pandas as pd
import numpy as np
import warnings


def count_percentage(category: str, temp_df: pd.DataFrame):
    '''
        category - название колонки, процент которой будет вычисляться
        temp_df - временный датасет только с двумся колонками: развитие_опп, одина из ['сахарный_диабет', 'гб', 'наличие_хбп']
    '''

    # преобразование датасета, чтобы остались только те, у кого есть хроническое заболевание
    temp_df[category] = temp_df[category].apply(lambda x: 1 if x == 1 else np.NaN)
    temp_df = temp_df.dropna(axis=0)

    # подсчёт количества пациентов, которые имеют/не имеют ОПП
    data = temp_df.groupby(['развитие_опп']).agg({category: 'count'})

    # высчитывание процента пациентов, которые имеют/не имеют ОПП
    answer = [category]
    summ = sum(list(data[category]))
    for en, j in enumerate(data[category]):
        answer.append(f'{list(data[category].index)[en]} ОПП - {round(j / summ * 100, 2)}%')

    '''
        возвращаем список, который содержит следующую информацию:
        answer[0] - название хронической болезни
        answer[1] - процент пациентов, которые болеют хронически и имеют ОПП
        answer[2] - процент пациентов, которые болеют хронически, но не имеют ОПП
    '''
    return answer


warnings.filterwarnings('ignore')

# загрузка и предобработка данных
df = pd.read_csv('/content/Content/processed_data.csv', skipinitialspace=True, sep=',')
df.columns = [i.replace(' ', '_').lower() for i in df.columns]

# создание новой колонки с бинарным типом данных
df['наличие_хбп'] = df['хбп'].apply(lambda x: 0 if x == 'Пациенты без ХБП' else 1)

# цикл по всем необходимым хроническим болезням
for categ in ['сахарный_диабет', 'гб', 'наличие_хбп']:
    print(*count_percentage(categ, df[[categ, 'развитие_опп']]))


сахарный_диабет есть ОПП - 69.49% нет ОПП - 30.51%
гб есть ОПП - 61.9% нет ОПП - 38.1%
наличие_хбп есть ОПП - 65.69% нет ОПП - 34.31%


# **3.6 Проверить адекватность поставленного диагноза по стадии хронической болезни почек (найти параметр, по которому ставится диагноз, использовать данные из внешних источников)**

In [9]:
# ЗАДАНИЕ 3.6
import pandas as pd


def make_prediction_to_diagnosis(skf):
    # устанавливаем стадию хбп у пациента
    if g1[0] <= skf <= g1[1]:
        return 'Стадия C1-C2'
    elif g2[1] < skf < g1[0]:
        return 'Стадия C1-С2'
    elif skf <= g2[1]:
        return 'Стадия C3'
    return 'Пациенты без ХБП'


def check_diagnosis(cols):
    # проверяем адекватность поставлxенного диагноза
    vals = cols.values
    if vals[0] == vals[1]:
        return 'Диагноз поставлен правильно'
    return 'Диагноз возможно поставлен некорректно'


# Загружаем и делаем предобработку данных
# скф_расч., 90-120 (параметр, по которому определяется стадия хбп)
df = pd.read_csv('/content/Content/processed_data.csv', skipinitialspace=True, decimal=',', sep=',')
df.columns = [i.replace(' ', '_').lower() for i in df.columns]

data = df[['развитие_опп', 'хбп', 'скф_расч']]
data['скф_расч'] = data['скф_расч'].astype(float)
# удаляем всех пациентов у которых нет опп
data = data[(data['развитие_опп'] == 'есть')]

# узнаем примерные рамки параметра скф по поставленной стадии
d = round(data.groupby(['хбп']).agg({'скф_расч': 'describe'})['скф_расч'], 2)
twenty_five = d['25%'].values
mean = d['mean'].values
seventy_five = d['75%'].values

g1 = [twenty_five[0], seventy_five[0]]
g2 = [twenty_five[1], seventy_five[1]]

# делаем колонку предположительного диагноза
data['предположительный_диагноз'] = data['скф_расч'].apply(lambda x: make_prediction_to_diagnosis(x))
# сравниваем колонку преположительного диагноза и хбп.
# Если диагноз подтверждается: Диагноз поставлен правильно
# Иначе: Диагноз возможно поставлен некорректно
data['адекватность_диагноза'] = data[['предположительный_диагноз', 'хбп']].apply(lambda x: check_diagnosis(x), axis=1)

print(data[['адекватность_диагноза', 'хбп']].head(10))


                     адекватность_диагноза               хбп
1   Диагноз возможно поставлен некорректно      Стадия C1-C2
4              Диагноз поставлен правильно      Стадия C1-C2
6   Диагноз возможно поставлен некорректно      Стадия C1-C2
8              Диагноз поставлен правильно      Стадия C1-C2
10             Диагноз поставлен правильно      Стадия C1-C2
13             Диагноз поставлен правильно      Стадия C1-C2
15  Диагноз возможно поставлен некорректно      Стадия C1-C2
16  Диагноз возможно поставлен некорректно  Пациенты без ХБП
17             Диагноз поставлен правильно      Стадия C1-C2
19             Диагноз поставлен правильно      Стадия C1-C2


# **3.7 Исследовать корреляцию между параметрами. Для наиболее сильных корреляций обосновать, имеет ли это реальный смысл или же просто особенность данных**

In [11]:
# ЗАДАНИЕ 3.7

import pandas as pd
from scipy.stats import pearsonr, spearmanr, mannwhitneyu, chi2_contingency, shapiro


def chi2_counting(first_cat, second_cat):
    # функция для подсчёта корреляции номинальных данных с номинальными

    temp_table = pd.crosstab(data[first_cat], data[second_cat])
    chi_2 = chi2_contingency(temp_table)
    return chi_2.statistic, chi_2.pvalue


def mannwhitneyu_counting(first_cat, second_cat):
    # функция для подсчёта корреляции количественных данных с номинальными

    mn = mannwhitneyu(first_cat, second_cat)
    cor, p_value = mn.statistic, mn.pvalue
    return cor, p_value


def pearsonr_counting(first_cat, second_cat):
    # функция для подсчёта корреляции количественных данных с количественными (с нормальным распределением данных)

    pr = pearsonr(first_cat, second_cat)
    cor, p_value = pr.statistic, pr.pvalue
    return cor, p_value


def spearmanr_counting(first_cat, second_cat):
    # функция для подсчёта корреляции количественных данных с количественными (с ненормальным распределением данных)

    sp = spearmanr(first_cat, second_cat)
    cor, p_value = sp.statistic, sp.pvalue
    return cor, p_value


data = pd.read_csv('/content/Content/processed_data.csv', skipinitialspace=True)
data.columns = [i.replace(' ', '_').lower() for i in data.columns]
nominal = ['развитие_опп', 'пол', 'гб', 'сахарный_диабет', 'стенокардия', 'инфаркт_миокарда', 'мерцательная_аритмия',
           'желудочковая_экстрасистолия', 'а-в_блокада', 'блокада_ножек_пучка_гиса', 'хсн', 'нк', 'ар', 'аик', 'хбп']

answer = list()

for first_cat in data.columns:
    for second_cat in data.columns:
        if first_cat != second_cat:
            g1 = data[first_cat]
            g2 = data[second_cat]
            if first_cat in nominal and second_cat in nominal:
                # высчитывание корреляции для номинальные-номинальные
                cor, p_value = chi2_counting(first_cat, second_cat)
                answer.append([first_cat, second_cat, round(cor, 2), round(p_value, 3)])
            elif first_cat in nominal and second_cat not in nominal or first_cat not in nominal and second_cat in nominal:
                # высчитывание корреляции для количественные-номинальные
                try:
                    cor, p_value = mannwhitneyu_counting(g1, g2)
                    answer.append([first_cat, second_cat, round(cor, 2), round(p_value, 3)])
                except TypeError:
                    pass
            else:
                # высчитывание корреляции для количественные-количественные
                if shapiro(g1).pvalue >= 0.05 and shapiro(g2).pvalue >= 0.05:
                    # Данные распределены нормально
                    cor, p_value = pearsonr_counting(g1, g2)
                    answer.append([first_cat, second_cat, round(cor, 2), round(p_value, 3)])
                else:
                    # Данные распределены ненормально
                    cor, p_value = spearmanr_counting(g1, g2)
                    answer.append([first_cat, second_cat, round(cor, 2), round(p_value, 3)])

# фильтруем данные
new_answer = list()
for result in answer:
    if result[-2] != 0 and -1 <= result[-2] <= 1 and result[-1] <= 0.05:

        new_answer.append(result)

answer_df = pd.DataFrame(data=sorted(new_answer, key=lambda x: x[-2], reverse=True),
                         columns=['Показатель №1', 'Показатель №2', 'Значение корреляции', 'p-уровень'])

print(answer_df.head(10))


           Показатель №1          Показатель №2  Значение корреляции  \
0                     уо                     мо                 0.99   
1                     мо                     уо                 0.99   
2                   нсо3                общ.со2                 0.98   
3                общ.со2                   нсо3                 0.98   
4                  дд_лж                    кдо                 0.93   
5                  сд_лж                    ксо                 0.93   
6                    кдо                  дд_лж                 0.93   
7                    ксо                  сд_лж                 0.93   
8       длительность_аик  время_пережатия_аорты                 0.89   
9  время_пережатия_аорты       длительность_аик                 0.89   

   p-уровень  
0        0.0  
1        0.0  
2        0.0  
3        0.0  
4        0.0  
5        0.0  
6        0.0  
7        0.0  
8        0.0  
9        0.0  


Как можно заметить что p-уровень у самых сильных корреляций равен 0, что может значить, что данный результат не имеет реального смысла и скорее это просто особенность данных